## Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from functools import partial

from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# Rich setup for Jupyter
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.aruco.sheet_aruco_config import SheetArucoConfig
from snap_fit.config.types import EdgePos
from snap_fit.data_models.piece_id import PieceId
from snap_fit.grid import GridModel
from snap_fit.grid import GridPos
from snap_fit.grid import Orientation
from snap_fit.grid import OrientedPieceType
from snap_fit.grid import PieceType
from snap_fit.grid import PlacementState
from snap_fit.grid import compute_rotation
from snap_fit.grid import score_edge
from snap_fit.grid import score_grid
from snap_fit.grid import score_grid_with_details
from snap_fit.image.utils import show_image_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece_matcher import PieceMatcher
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager

## Load Puzzle Sheets


In [ ]:
# Configure ArUco (matching sample_puzzle_v2 config)
board_config = ArucoBoardConfig(
    markers_x=7,
    markers_y=5,
    marker_length=100,
    marker_separation=100,
)
detector_config = ArucoDetectorConfig(board=board_config)
sheet_config = SheetArucoConfig(detector=detector_config)
sheet_aruco = SheetAruco(sheet_config)

# Loader function
aruco_loader = sheet_aruco.load_sheet

# Data folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "sample_puzzle_v2" / "sheets"
lg.info(f"Loading data from {data_dir}")

In [ ]:
# Load sheets
manager = SheetManager()
manager.add_sheets(
    folder_path=data_dir,
    pattern="*.png",
    loader_func=aruco_loader,
)

pieces = manager.get_pieces_ls()
rprint(f"Loaded {len(pieces)} pieces from {len(manager.get_sheets_ls())} sheets")

## Examine Piece OrientedPieceType

Each piece now has an `oriented_piece_type` derived from its flat edges.


In [ ]:
# Group pieces by type
corners = [p for p in pieces if p.oriented_piece_type.piece_type == PieceType.CORNER]
edges = [p for p in pieces if p.oriented_piece_type.piece_type == PieceType.EDGE]
inners = [p for p in pieces if p.oriented_piece_type.piece_type == PieceType.INNER]

rprint(f"Corners: {len(corners)}")
rprint(f"Edges: {len(edges)}")
rprint(f"Inners: {len(inners)}")

In [ ]:
# Examine a few pieces
rprint("[bold]Sample pieces and their types:[/bold]")
for piece in pieces[:10]:
    opt = piece.oriented_piece_type
    flat_str = (
        ", ".join(e.name for e in piece.flat_edges) if piece.flat_edges else "none"
    )
    rprint(f"  {piece.piece_id}: {opt} | flat edges: {flat_str}")

In [ ]:
# Show a corner piece
if corners:
    corner = corners[0]
    rprint(f"Corner piece: {corner.piece_id}")
    rprint(f"  Type: {corner.oriented_piece_type}")
    rprint(f"  Flat edges: {corner.flat_edges}")
    show_image_mpl(corner.img_orig, figsize=(4, 4))

## Test get_segment_at

Get segments considering rotation.


In [ ]:
# Pick a piece
test_piece = pieces[0]
rprint(f"Test piece: {test_piece.piece_id}")

# Get segment at TOP with no rotation
seg_top_0 = test_piece.get_segment_at(EdgePos.TOP, Orientation.DEG_0)
rprint(f"Segment at TOP (no rotation): shape={seg_top_0.shape}")

# Get segment at TOP after 90° rotation
# This should return what was originally at LEFT
seg_top_90 = test_piece.get_segment_at(EdgePos.TOP, Orientation.DEG_90)
rprint(f"Segment at TOP (90° rotation): shape={seg_top_90.shape}")

# Verify: should be same as original LEFT segment
seg_left_orig = test_piece.segments[EdgePos.LEFT]
rprint(f"Original LEFT segment: shape={seg_left_orig.shape}")
rprint(f"Same segment? {seg_top_90 is seg_left_orig}")

## Create Grid and Place Pieces


In [ ]:
# sample_puzzle_v2 is 8x6 (tiles_x=8, tiles_y=6)
# But we'll use a smaller grid for demo: 3x4
grid = GridModel(rows=3, cols=4)
rprint(f"Grid: {grid}")
rprint(
    f"Need: {len(grid.corners)} corners, {len(grid.edges)} edges, {len(grid.inners)} inners"
)

In [ ]:
# Create placement state
state = PlacementState(grid)

# Place some pieces (we'll just grab pieces without worrying about correct placement)
# In a real solver, we'd match types and compute rotations

# Place corners
for i, pos in enumerate(grid.corners):
    if i < len(corners):
        piece = corners[i]
        slot = grid.get_slot_type(pos)
        rotation = compute_rotation(piece.oriented_piece_type, slot)
        state.place(piece.piece_id, pos, rotation)
        rprint(
            f"Placed corner {piece.piece_id} at {pos} with rotation {rotation.value}°"
        )

In [ ]:
# Place edges
for i, pos in enumerate(grid.edges):
    if i < len(edges):
        piece = edges[i]
        slot = grid.get_slot_type(pos)
        rotation = compute_rotation(piece.oriented_piece_type, slot)
        state.place(piece.piece_id, pos, rotation)

rprint(f"Placed {min(len(edges), len(grid.edges))} edge pieces")

In [ ]:
# Place inners
for i, pos in enumerate(grid.inners):
    if i < len(inners):
        piece = inners[i]
        slot = grid.get_slot_type(pos)
        rotation = compute_rotation(piece.oriented_piece_type, slot)
        state.place(piece.piece_id, pos, rotation)

rprint(f"Placed {min(len(inners), len(grid.inners))} inner pieces")
rprint(f"\nFinal state: {state}")

## Initialize Matcher and Score


In [ ]:
# Create matcher
matcher = PieceMatcher(manager)
rprint(f"Matcher ready, cache empty: {len(matcher._lookup)} entries")

In [ ]:
# Score single edge
pos1 = GridPos(ro=0, co=0)
pos2 = GridPos(ro=0, co=1)

edge_score = score_edge(state, pos1, pos2, matcher)
rprint(f"Score between {pos1} and {pos2}: {edge_score}")

In [ ]:
# Score entire grid
total_score = score_grid(state, matcher)
rprint(f"[bold]Total grid score: {total_score:.2f}[/bold]")
rprint(f"Cache size after scoring: {len(matcher._lookup)} entries")

In [ ]:
# Detailed scoring
total, edge_scores = score_grid_with_details(state, matcher)

rprint("[bold]Edge-by-edge scores:[/bold]")
for (p1, p2), score in sorted(edge_scores.items(), key=lambda x: x[1])[:10]:
    rprint(f"  {p1} <-> {p2}: {score:.2f}")

rprint(f"\nBest match: {min(edge_scores.values()):.2f}")
rprint(f"Worst match: {max(edge_scores.values()):.2f}")
rprint(f"Average: {sum(edge_scores.values()) / len(edge_scores):.2f}")

## Test Cached Score Lookup


In [ ]:
# Get a segment pair that was scored
from snap_fit.data_models.segment_id import SegmentId

# Pick two adjacent pieces
placement1 = state.get_placement(GridPos(ro=0, co=0))
placement2 = state.get_placement(GridPos(ro=0, co=1))

if placement1 and placement2:
    piece_id1, rot1 = placement1
    piece_id2, rot2 = placement2

    # Build segment IDs (right edge of piece1, left edge of piece2)
    from snap_fit.grid.orientation_utils import get_original_edge_pos

    orig_edge1 = get_original_edge_pos(EdgePos.RIGHT, rot1)
    orig_edge2 = get_original_edge_pos(EdgePos.LEFT, rot2)

    seg_id1 = SegmentId(piece_id=piece_id1, edge_pos=orig_edge1)
    seg_id2 = SegmentId(piece_id=piece_id2, edge_pos=orig_edge2)

    # Check if cached
    cached = matcher.get_cached_score(seg_id1, seg_id2)
    rprint(f"Cached score for {seg_id1} <-> {seg_id2}: {cached}")

## Summary

The grid scoring system works end-to-end:

1. **Pieces have `OrientedPieceType`** - Automatically derived from flat edges
2. **`get_segment_at()` works** - Returns correct segment considering rotation
3. **`compute_rotation()` aligns pieces** - Calculates rotation to fit slot
4. **`score_edge()` and `score_grid()`** - Use matcher cache efficiently
5. **`get_cached_score()`** - Public access to matcher cache

The system is ready for solver integration!
